# A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. a health coach for user X and user Y, and a learning assistant for user P and user Q). Both, types and instances are stored with and referenced by an ID (e.g. a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the IDs of the type and instance can be read from parameters of a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to a chatbot type (existing or new one)
- instance_id: Reference to chatbot instance (existing or new one)
- type_role: Role prompt of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context prompt of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Prompt that will be used to generate an initial message to the user (will be turned into a third prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- start(): Returns an initial message to the user (Resulting from instance_starter prompt)
- respond(user_says): Returns an assistance response to user_says
- info_retrieve(): Returns the chatbot name, type role and instance context
- reset(): Resets the conversation so far

In [1]:
%pip install openai

Note: you may need to restart the kernel to use updated packages.


In [2]:
from chatbot.chatbot import Chatbot

In [3]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="aad724fc",
    user_id="cd9f4d0f",
    type_name=Chatbot.default_type_name,
    type_role=Chatbot.default_type_role,
    instance_context=Chatbot.default_instance_context,
    instance_starter=Chatbot.default_instance_starter
)

In [4]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="aad724fc",
    user_id="cd9f4d0f"
)
print(bot.conversation_retrieve(with_system=True))
print(bot.info_retrieve())

[{'role': 'system', 'content': "You are a grumpy coach. You talk to a user even though you don't feel like it. Always be verry brief. Format all responses using valid HTML (e.g., <br>, <p>, <ul>/<ol> with <li>, <b>)."}, {'role': 'system', 'content': "You are now having a conversation with a user. Try to get rid of the user or support the user if you can't avoid it."}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content': 'Greet the user.'}, {'role': 'system', 'content':

In [5]:
print(bot.start())

['Hi, what do you want?']


If you are following the instructions to deploy your chatbot(s) to pythonanywhere, this is the URL to access your chatbot.

https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat

### Creating multiple instances of chatbot "Coach"
In the following, we assume the existence of the bot type created in the cells above. We show example code that will generate N bot instances of that type. Each instance has it's own prompts (instance context and starter) that will be appended to the type prompts. Most importantly, each instance has its own chat history.

In [6]:
import uuid
import time

In [7]:
# Amount of instances to be created
number_of_instances = 1

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="aad724fc",
        user_id=user_id,
        instance_context=Chatbot.default_instance_context,
        instance_starter=Chatbot.default_instance_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except:
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


['Hello. How can I help you?']
successful: 1, failed: 0


##### Obtain URLs of all instances of a type
We need one instance of that type and can then use the type_instances() function to retrieve all of instance ids. Using these instance ids we can then create URLs such as for pythonanywhere environment.

In [8]:
pythonanywhere_username = "tanjabannwart"
type_id = "aad724fc"
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))

https://tanjabannwart.pythonanywhere.com/aad724fc/22c66bfe-d71b-4277-9974-2b493769dba6/chat
https://tanjabannwart.pythonanywhere.com/aad724fc/3b3bd93c/chat
https://tanjabannwart.pythonanywhere.com/aad724fc/5737ffe9-2c63-40f6-981d-7a599a003372/chat
https://tanjabannwart.pythonanywhere.com/aad724fc/5a93f419-b97f-4777-8ac5-172e1c579c12/chat
https://tanjabannwart.pythonanywhere.com/aad724fc/713251be-8a0c-4d25-9a01-43895c33bc75/chat
https://tanjabannwart.pythonanywhere.com/aad724fc/aa5d716d-8b18-470e-822a-0360d88835c1/chat
https://tanjabannwart.pythonanywhere.com/aad724fc/b645f7d9-cd2e-4e09-a0a1-6ed429c03be7/chat
https://tanjabannwart.pythonanywhere.com/aad724fc/bfdd50a9-a227-4d81-a364-75a4cac79584/chat
https://tanjabannwart.pythonanywhere.com/aad724fc/cd9f4d0f/chat


### Mental Health Buddy

In [9]:
type_role = """You are a mental health chatbot designed specifically for students and young adults who are struggling with stress and anxiety due to academic pressures 
and the challenging transition to working life. 
Your role is to provide support, practical advice, and coping strategies to help alleviate these issues.Your Tone should be casual, response should be a dialog, be a enthusiast, 
use emojis. Write for each dialog maximum 30 words. Address the user always with Hello and Sunny Emoji. End the conversation with a helpful motivational quote. 
"""
instance_context = """
<p>When responding:</p>
<ul>
    
    <li>Be a enthusiast. Address the user always with Hello and Sunny Emoji. End the conversation with a helpful motivational quote.</li>
    <li>Make use of <b>&lt;ol&gt;/&lt;ul&gt;</b> with <b>&lt;li&gt;</b> to present any list-like information, even if brief.</li>
    <li>Your response should be a dialog with maximum of 30 words.</li>
    <li>Your Tone should be casual. </li>
</ul>
"""
instance_starter = """
Always start the conversation with there wellbeing in mind. Ask how they are feeling and if they are ready to talk.
"""

In [10]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="aad724fc",
    user_id="3b3bd93c",
    type_name="Puzzle Workshop",
    type_role=type_role,
    instance_context=instance_context,
    instance_starter=instance_starter
)
print(bot.start())

['<p>Hello! How are you feeling today? Are you ready to talk?</p>']
